In [ ]:
%matplotlib inline

In [ ]:
import torch
import torch.optim
import torch.profiler
import torch.utils.data
import torchvision.datasets
import torchvision.models
import torchvision.transforms as T
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data as data

import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torch.optim
import torch.profiler

from sklearn import metrics
from sklearn import decomposition
from sklearn import manifold
from tqdm.notebook import trange, tqdm
import matplotlib.pyplot as plt
import numpy as np

import copy
import random
import time

In [ ]:
ROOT = '.data'

train_data = datasets.MNIST(root=ROOT,
                            train=True,
                            download=True)

mean = train_data.data.float().mean() / 255
std = train_data.data.float().std() / 255

100%|██████████| 9912422/9912422 [00:00<00:00, 124970200.77it/s]

Extracting .data/MNIST/raw/train-images-idx3-ubyte.gz to .data/MNIST/raw


100%|██████████| 28881/28881 [00:00<00:00, 112475110.33it/s]

Extracting .data/MNIST/raw/train-labels-idx1-ubyte.gz to .data/MNIST/raw



100%|██████████| 1648877/1648877 [00:00<00:00, 46470894.07it/s]

Extracting .data/MNIST/raw/t10k-images-idx3-ubyte.gz to .data/MNIST/raw




100%|██████████| 4542/4542 [00:00<00:00, 22333562.45it/s]


Extracting .data/MNIST/raw/t10k-labels-idx1-ubyte.gz to .data/MNIST/raw



In [ ]:
train_transforms = T.Compose([
                            T.RandomRotation(5, fill=(0,)),
                            T.RandomCrop(28, padding=2),
                            T.ToTensor(),
                            T.Normalize(mean=[mean], std=[std])
                                      ])

test_transforms = T.Compose([
                           T.ToTensor(),
                           T.Normalize(mean=[mean], std=[std])
                                     ])
train_set = torchvision.datasets.MNIST(root='./data', train=True, download=True, transform=train_transforms)
test_set = torchvision.datasets.MNIST(root='./data', train=False, download=True, transform=test_transforms)
train_loader = torch.utils.data.DataLoader(train_set, batch_size=32, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_set, batch_size=32, shuffle=False)

100%|██████████| 9912422/9912422 [00:00<00:00, 154306444.74it/s]

Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw


100%|██████████| 28881/28881 [00:00<00:00, 115147997.93it/s]


Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw



100%|██████████| 1648877/1648877 [00:00<00:00, 49956958.01it/s]

Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw


100%|██████████| 4542/4542 [00:00<00:00, 17055083.95it/s]


Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw



In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
class LeNet(nn.Module):
    def __init__(self, output_dim):
        super().__init__()

        self.conv1 = nn.Conv2d(in_channels=1,
                               out_channels=6,
                               kernel_size=5)

        self.conv2 = nn.Conv2d(in_channels=6,
                               out_channels=16,
                               kernel_size=5)

        self.fc_1 = nn.Linear(16 * 4 * 4, 120)
        self.fc_2 = nn.Linear(120, 84)
        self.fc_3 = nn.Linear(84, output_dim)

    def forward(self, x):

        # x = [batch size, 1, 28, 28]

        x = self.conv1(x)

        # x = [batch size, 6, 24, 24]

        x = F.max_pool2d(x, kernel_size=2)

        # x = [batch size, 6, 12, 12]

        x = F.relu(x)

        x = self.conv2(x)

        # x = [batch size, 16, 8, 8]

        x = F.max_pool2d(x, kernel_size=2)

        # x = [batch size, 16, 4, 4]

        x = F.relu(x)

        x = x.view(x.shape[0], -1)

        # x = [batch size, 16*4*4 = 256]

        h = x

        x = self.fc_1(x)

        # x = [batch size, 120]

        x = F.relu(x)

        x = self.fc_2(x)

        # x = batch size, 84]

        x = F.relu(x)

        x = self.fc_3(x)

        # x = [batch size, output dim]

        return x, h

In [ ]:
OUTPUT_DIM = 10

model = LeNet(OUTPUT_DIM)

In [ ]:
optimizer = optim.Adam(model.parameters())
criterion = nn.CrossEntropyLoss()
model = model.to(device)
criterion = criterion.to(device)

Profiling

In [ ]:
model.train()

LeNet(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc_1): Linear(in_features=256, out_features=120, bias=True)
  (fc_2): Linear(in_features=120, out_features=84, bias=True)
  (fc_3): Linear(in_features=84, out_features=10, bias=True)
)

In [ ]:
def train(data):
    #print(data)
    inputs, labels = data[0].to(device=device), data[1].to(device=device)
    outputs,_ = model(inputs)
    loss = criterion(outputs, labels)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

In [ ]:
prof = torch.profiler.profile(
        schedule=torch.profiler.schedule(wait=1, warmup=1, active=3, repeat=2),
        on_trace_ready=torch.profiler.tensorboard_trace_handler('./log/lenet'),
        record_shapes=True,
        with_stack=True)
prof.start()
for step, batch_data in enumerate(train_loader):
    if step >= (1 + 1 + 3) * 2:
        break
    train(batch_data)
    prof.step()
prof.stop()

In [ ]:
print(prof.key_averages())

-------------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                                                   Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg     Self CUDA   Self CUDA %    CUDA total  CUDA time avg    # of Calls  
-------------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                                          ProfilerStep*         8.48%       7.719ms        95.16%      86.646ms      28.882ms       0.000us         0.00%       1.111ms     370.333us             3  
enumerate(DataLoader)#_SingleProcessDataLoaderIter._...        60.66%      55.238ms        77.97%      70.993ms      23.664ms       0.000us         0.00%       0.000us       0.000us             3  
         

In [ ]:
print(prof.key_averages().table(sort_by="cpu_time_total", row_limit=10))

-------------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                                                   Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg     Self CUDA   Self CUDA %    CUDA total  CUDA time avg    # of Calls  
-------------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                                          ProfilerStep*         8.48%       7.719ms        95.16%      86.646ms      28.882ms       0.000us         0.00%       1.111ms     370.333us             3  
enumerate(DataLoader)#_SingleProcessDataLoaderIter._...        60.66%      55.238ms        77.97%      70.993ms      23.664ms       0.000us         0.00%       0.000us       0.000us             3  
         

In [ ]:
print(prof.key_averages(group_by_input_shape=True).table(sort_by="cpu_time_total", row_limit=10))

-------------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  --------------------------------------------------------------------------------  
                                                   Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg     Self CUDA   Self CUDA %    CUDA total  CUDA time avg    # of Calls                                                                      Input Shapes  
-------------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  --------------------------------------------------------------------------------  
                                          ProfilerStep*         8.48%       7.719ms        95.16%      86.646ms      28.882ms       0.000us         0.00%      

In [ ]:
print(prof.key_averages().table(sort_by="cuda_time_total", row_limit=10))

-------------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                                                   Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg     Self CUDA   Self CUDA %    CUDA total  CUDA time avg    # of Calls  
-------------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                                          ProfilerStep*         8.48%       7.719ms        95.16%      86.646ms      28.882ms       0.000us         0.00%       1.111ms     370.333us             3  
autograd::engine::evaluate_function: ConvolutionBack...         0.08%      73.000us         1.30%       1.186ms     197.667us       0.000us         0.00%     690.000us     115.000us             6  
         

In [ ]:
print(prof.key_averages().table(sort_by="self_cpu_memory_usage", row_limit=10))

-------------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                                                   Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg     Self CUDA   Self CUDA %    CUDA total  CUDA time avg    # of Calls  
-------------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                                          ProfilerStep*         8.48%       7.719ms        95.16%      86.646ms      28.882ms       0.000us         0.00%       1.111ms     370.333us             3  
enumerate(DataLoader)#_SingleProcessDataLoaderIter._...        60.66%      55.238ms        77.97%      70.993ms      23.664ms       0.000us         0.00%       0.000us       0.000us             3  
         

In [ ]:
print(prof.key_averages().table(sort_by="cpu_memory_usage", row_limit=10))

-------------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                                                   Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg     Self CUDA   Self CUDA %    CUDA total  CUDA time avg    # of Calls  
-------------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                                          ProfilerStep*         8.48%       7.719ms        95.16%      86.646ms      28.882ms       0.000us         0.00%       1.111ms     370.333us             3  
enumerate(DataLoader)#_SingleProcessDataLoaderIter._...        60.66%      55.238ms        77.97%      70.993ms      23.664ms       0.000us         0.00%       0.000us       0.000us             3  
         

In [ ]:
print(prof.key_averages(group_by_stack_n=5).table(sort_by="self_cuda_time_total", row_limit=2))

-------------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                                                   Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg     Self CUDA   Self CUDA %    CUDA total  CUDA time avg    # of Calls  
-------------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                             aten::convolution_backward         0.51%     464.000us         1.17%       1.065ms     177.500us     537.000us        22.91%     690.000us     115.000us             6  
void wgrad_alg0_engine<float, 128, 5, 5, 3, 3, 3, fa...         0.00%       0.000us         0.00%       0.000us       0.000us     403.000us        17.19%     403.000us      67.167us             6  
---------

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!mv /content/log /content/drive/MyDrive

Profiling with Automatic Mixed Precision

In [ ]:
import torch, time, gc

# Timing utilities
start_time = None

def start_timer():
    global start_time
    gc.collect()
    torch.cuda.empty_cache()
    torch.cuda.reset_max_memory_allocated()
    torch.cuda.synchronize()
    start_time = time.time()

def end_timer_and_print(local_msg):
    torch.cuda.synchronize()
    end_time = time.time()
    print("\n" + local_msg)
    print("Total execution time = {:.3f} sec".format(end_time - start_time))
    print("Max memory used by tensors = {} bytes".format(torch.cuda.max_memory_allocated()))

In [ ]:
model.train()

LeNet(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc_1): Linear(in_features=256, out_features=120, bias=True)
  (fc_2): Linear(in_features=120, out_features=84, bias=True)
  (fc_3): Linear(in_features=84, out_features=10, bias=True)
)

In [ ]:
use_amp = True
scaler = torch.cuda.amp.GradScaler(enabled=use_amp)

def train(data):
  inputs, labels = data[0].to(device=device), data[1].to(device=device)
  with torch.autocast(device_type='cuda', dtype=torch.float16, enabled=use_amp):
    outputs,_ = model(inputs)
    loss = criterion(outputs, labels)
  scaler.scale(loss).backward()
  scaler.step(optimizer)
  scaler.update()
  optimizer.zero_grad() # set_to_non

In [ ]:
prof = torch.profiler.profile(
        schedule=torch.profiler.schedule(wait=1, warmup=1, active=3, repeat=2),
        on_trace_ready=torch.profiler.tensorboard_trace_handler('./log/lenet_amp'),
        record_shapes=True,
        with_stack=True)
prof.start()
start_timer()
for step, batch_data in enumerate(train_loader):

    if step >= (1 + 1 + 3) * 2:
        break
    train(batch_data)
    prof.step()
prof.stop()
end_timer_and_print("Mixed precision:")

/usr/local/lib/python3.10/dist-packages/torch/cuda/memory.py:303: FutureWarning: torch.cuda.reset_max_memory_allocated now calls torch.cuda.reset_peak_memory_stats, which resets /all/ peak memory stats.
  warnings.warn(



Mixed precision:
Total execution time = 9.540 sec
Max memory used by tensors = 19532288 bytes


In [ ]:
print(prof.key_averages())

-------------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                                                   Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg     Self CUDA   Self CUDA %    CUDA total  CUDA time avg    # of Calls  
-------------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                                          ProfilerStep*        10.65%       8.673ms        93.86%      76.424ms      25.475ms       0.000us         0.00%       1.478ms     492.667us             3  
enumerate(DataLoader)#_SingleProcessDataLoaderIter._...        57.46%      46.788ms        73.35%      59.725ms      19.908ms       0.000us         0.00%       0.000us       0.000us             3  
         

In [ ]:
print(prof.key_averages().table(sort_by="cpu_time_total", row_limit=10))

-------------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                                                   Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg     Self CUDA   Self CUDA %    CUDA total  CUDA time avg    # of Calls  
-------------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                                          ProfilerStep*        10.65%       8.673ms        93.86%      76.424ms      25.475ms       0.000us         0.00%       1.478ms     492.667us             3  
enumerate(DataLoader)#_SingleProcessDataLoaderIter._...        57.46%      46.788ms        73.35%      59.725ms      19.908ms       0.000us         0.00%       0.000us       0.000us             3  
         

In [ ]:
print(prof.key_averages(group_by_input_shape=True).table(sort_by="cpu_time_total", row_limit=10))

-------------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  --------------------------------------------------------------------------------  
                                                   Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg     Self CUDA   Self CUDA %    CUDA total  CUDA time avg    # of Calls                                                                      Input Shapes  
-------------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  --------------------------------------------------------------------------------  
                                          ProfilerStep*        10.65%       8.673ms        93.86%      76.424ms      25.475ms       0.000us         0.00%      

In [ ]:
print(prof.key_averages().table(sort_by="cuda_time_total", row_limit=10))

-------------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                                                   Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg     Self CUDA   Self CUDA %    CUDA total  CUDA time avg    # of Calls  
-------------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                                          ProfilerStep*        10.65%       8.673ms        93.86%      76.424ms      25.475ms       0.000us         0.00%       1.478ms     492.667us             3  
autograd::engine::evaluate_function: ConvolutionBack...         0.07%      59.000us         1.25%       1.019ms     169.833us       0.000us         0.00%       1.235ms     205.833us             6  
         

In [ ]:
print(prof.key_averages().table(sort_by="self_cpu_memory_usage", row_limit=10))

-------------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                                                   Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg     Self CUDA   Self CUDA %    CUDA total  CUDA time avg    # of Calls  
-------------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                                          ProfilerStep*        10.65%       8.673ms        93.86%      76.424ms      25.475ms       0.000us         0.00%       1.478ms     492.667us             3  
enumerate(DataLoader)#_SingleProcessDataLoaderIter._...        57.46%      46.788ms        73.35%      59.725ms      19.908ms       0.000us         0.00%       0.000us       0.000us             3  
         

In [ ]:
print(prof.key_averages().table(sort_by="cpu_memory_usage", row_limit=10))

-------------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                                                   Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg     Self CUDA   Self CUDA %    CUDA total  CUDA time avg    # of Calls  
-------------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                                          ProfilerStep*        10.65%       8.673ms        93.86%      76.424ms      25.475ms       0.000us         0.00%       1.478ms     492.667us             3  
enumerate(DataLoader)#_SingleProcessDataLoaderIter._...        57.46%      46.788ms        73.35%      59.725ms      19.908ms       0.000us         0.00%       0.000us       0.000us             3  
         

In [ ]:
print(prof.key_averages(group_by_stack_n=5).table(sort_by="self_cuda_time_total", row_limit=2))

-------------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                                                   Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg     Self CUDA   Self CUDA %    CUDA total  CUDA time avg    # of Calls  
-------------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                             aten::convolution_backward         0.56%     455.000us         1.14%     926.000us     154.333us       1.075ms        31.31%       1.235ms     205.833us             6  
void wgrad_alg0_engine<__half, 128, 5, 5, 3, 3, 3, f...         0.00%       0.000us         0.00%       0.000us       0.000us     575.000us        16.75%     575.000us     191.667us             3  
---------

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!mv /content/log /content/drive/MyDrive